In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/prediction/custom_prediction_routines/SDK_Triton_PyTorch_Local_Prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/prediction/custom_prediction_routines/SDK_Triton_PyTorch_Local_Prediction.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>

## 概述

本教程演示了如何使用Vertex AI SDK本地测试[NVIDIA Triton推理服务器](https://developer.nvidia.com/nvidia-triton-inference-server)，以提供PyTorch模型并将其部署到Vertex AI Predictions。


### 数据集

本教程使用R.A. Fisher的鸢尾花数据集，这是一个小数据集，非常流行用于尝试机器学习技术。每个实例具有四个数值特征，这些特征是对花的不同测量，以及一个目标标签，将其标记为三种类型的鸢尾花之一：山鸢尾、变色鸢尾或维吉尼亚鸢尾。

本教程使用[鸢尾花数据集](https://archive.ics.uci.edu/ml/machine-learning-databases/iris)。


### 目标

目标是：
- 使用花的测量作为输入训练一个PyTorch模型，以预测其是什么类型的鸢尾花。
- 保存模型。
- 本地测试NVIDIA Triton推理服务器。
- 将Triton推理服务器作为自定义容器上传并部署到Vertex Prediction。

本教程更侧重于使用Vertex AI部署此模型，而不是设计模型本身。


### 成本

本教程使用Google Cloud的收费组件：

* Vertex AI

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

### 设置您的本地开发环境

**如果您正在使用Vertex AI工作台笔记本**，则您的环境已满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境符合此笔记本的要求。
您需要以下内容：

* Docker
* Git
* Google Cloud SDK （gcloud）
* Python 3
* virtualenv
* 在使用 Python 3 的虚拟环境中运行的 Jupyter 笔记本

Google Cloud 的 [设置 Python 开发环境指南](https://cloud.google.com/python/setup) 和 [Jupyter 安装指南](https://jupyter.org/install) 提供了满足这些要求的详细说明。以下步骤提供了一套简洁的说明：

1. [安装并初始化 Cloud SDK。](https://cloud.google.com/sdk/docs/)

1. [安装 Python 3。](https://cloud.google.com/python/setup#installing_python)

1. [安装 virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) 并创建一个使用 Python 3 的虚拟环境。激活虚拟环境。

1. 要安装 Jupyter，在终端 shell 中的命令行中运行 `pip install jupyter`。

1. 要启动 Jupyter，在终端 shell 的命令行中运行 `jupyter notebook`。

1. 在 Jupyter Notebook 仪表板中打开此笔记本。

### 安装额外的包

安装在您的笔记本环境中未安装的附加包依赖项，例如 NumPy、Scikit-learn、FastAPI、Uvicorn 和 joblib。

In [ ]:
%%writefile requirements.txt
pandas
torch==1.11.0
google-cloud-storage>=1.26.0,<2.0.0dev
google-cloud-aiplatform[prediction]>=1.16.0

你部署的模型将预先安装了一组与笔记本环境不同的依赖项。你不应该假设因为笔记本里的功能正常运行，它们也会在模型中正常工作。相反，你应该通过在requirements.txt中列出依赖项，然后使用`pip install`来在笔记本中安装完全相同的依赖项。当然，请注意，有可能你在requirements.txt中漏掉了笔记本中已存在的依赖项。如果是这种情况，功能将在笔记本中运行，但在模型中不会。为了防范这种情况，在部署到云端之前，你将在本地测试模型。

In [ ]:
# Install the same dependencies used in the serving container in the notebook
# environment.
%pip install -U --user -r requirements.txt

重新启动内核

在安装了额外包之后，你需要重新启动笔记本的内核，这样它才能找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在你开始之前

### 在这个示例中设置日志记录

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

### 设置您的Google Cloud项目

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用用于计算/存储成本。

1. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

1. 如果您是在本地运行此笔记本，则需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格以确保
Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会将以`!`或`%`开头的行视为shell命令，并将Python变量插入这些命令中的`$`或`{}`。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用 `gcloud` 获取您的项目ID。

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

否则，请在这里设置您的项目ID。

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
! gcloud config set project $PROJECT_ID

## 区域

您还可以更改 `REGION` 变量，该变量用于此笔记本其余部分的操作。以下是Vertex AI支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能不会使用多区域存储桶来训练Vertex AI。并非所有区域都支持所有Vertex AI服务。

了解有关[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

###配置项目和资源名称

如果您在直播教程会话中，您可能会使用一个共享的测试帐户或项目。为了避免用户在创建的资源之间发生名称冲突，您需要为每个实例会话创建一个时间戳，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

配置GCP资源名称。

In [ ]:
MODEL_ARTIFACT_DIR = "triton-pytorch"  # @param {type:"string"}
REPOSITORY = "custom-container-prediction-sdk"  # @param {type:"string"}
IMAGE = "triton-pytorch"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "triton-pytorch"  # @param {type:"string"}

`MODEL_ARTIFACT_DIR` - 在云存储桶中指向模型工件的文件夹目录路径，例如："my-models/fraud-detection/trial-4"

`REPOSITORY` - 要创建或使用的工件存储库的名称。

`IMAGE` - 将要推送的容器映像的名称。

`MODEL_DISPLAY_NAME` - Vertex AI 模型资源的显示名称。

### 创建一个云存储桶

**无论你使用什么笔记本环境，都需要按照以下步骤进行。**

如果要更新模型工件而不重新构建容器，您必须将模型工件和任何自定义代码上传到云存储中。

在下面设置您的云存储桶的名称。它必须在所有云存储桶中是唯一的。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "-aip-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

只有在您的存储桶不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

建立目录.

决定放置模型工件的目录。

In [ ]:
LOCAL_MODEL_ARTIFACTS_DIR = "model_artifacts"  # @param {type:"string"}

In [ ]:
%mkdir $LOCAL_MODEL_ARTIFACTS_DIR

## 训练一个 PyTorch 模型

### 下载鸢尾花数据
在这个例子中，你想要为简单的[鸢尾花数据集](https://archive.ics.uci.edu/ml/datasets/iris)构建一个分类器。因此，首先你需要将数据的 csv 文件下载到本地。

In [ ]:
DATA_DIR = "data_iris"

%mkdir $DATA_DIR

LOCAL_DATA_FILE = f"{DATA_DIR}/iris.csv"

In [ ]:
from urllib.request import urlretrieve

urlretrieve(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data",
    LOCAL_DATA_FILE,
)

构建一个PyTorch神经网络分类器

确保已安装PyTorch软件包。[点击此处安装](https://pytorch.org/get-started/locally/)。

In [ ]:
import torch
from torch.autograd import Variable

print("PyTorch Version: {}".format(torch.__version__))

#### 步骤 1. 加载数据

在这一步中，你需要：
1. 将数据加载到 Pandas 数据框中。
2. 将类别特征（种类）从字符串转换为数值指示器。
3. 将数据框分割为输入特征（xtrain）和目标特征（ytrain）。

In [ ]:
import pandas as pd

CLASS_VOCAB = ["setosa", "versicolor", "virginica"]

datatrain = pd.read_csv(
    LOCAL_DATA_FILE,
    names=["sepal_length", "sepal_width", "petal_length", "petal_width", "species"],
)

# change string value to numeric
datatrain.loc[datatrain["species"] == "Iris-setosa", "species"] = 0
datatrain.loc[datatrain["species"] == "Iris-versicolor", "species"] = 1
datatrain.loc[datatrain["species"] == "Iris-virginica", "species"] = 2
datatrain = datatrain.apply(pd.to_numeric)

# change dataframe to array
datatrain_array = datatrain.values

# split x and y (feature and target)
xtrain = datatrain_array[:, :4]
ytrain = datatrain_array[:, 4]

input_features = xtrain.shape[1]
num_classes = len(CLASS_VOCAB)

print("Records loaded: {}".format(len(xtrain)))
print("Number of input features: {}".format(input_features))
print("Number of classes: {}".format(num_classes))

第二步。设置模型参数
您可以尝试不同的值来调整**hidden_units**或**learning_rate**。

In [ ]:
HIDDEN_UNITS = 10
LEARNING_RATE = 0.1

步骤3. 定义PyTorch神经网络模型
在这里，您建立一个具有一个隐藏层和一个Softmax输出层用于分类的神经网络。

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(input_features, HIDDEN_UNITS),
    torch.nn.Sigmoid(),
    torch.nn.Linear(HIDDEN_UNITS, num_classes),
    torch.nn.Softmax(),
)

loss_metric = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

步骤4. 训练模型
您将训练模型进行 **num_epoch** 个周期。

In [ ]:
NUM_EPOCHS = 10000

for epoch in range(NUM_EPOCHS):

    x = Variable(torch.Tensor(xtrain).float())
    y = Variable(torch.Tensor(ytrain).long())
    optimizer.zero_grad()
    y_pred = model(x)
    loss = loss_metric(y_pred, y)
    loss.backward()
    optimizer.step()
    if (epoch) % 1000 == 0:
        print(
            "Epoch [{}/{}] Loss: {}".format(
                epoch + 1, NUM_EPOCHS, round(loss.item(), 3)
            )
        )

print("Epoch [{}/{}] Loss: {}".format(epoch + 1, NUM_EPOCHS, round(loss.item(), 3)))

步骤5. 保存模型

Triton推理服务器要求将PyTorch模型保存为[TorchScript格式](https://pytorch.org/tutorials/beginner/Intro_to_TorchScript_tutorial.html)。

In [ ]:
# An example input you would normally provide to your model's forward() method.
example = torch.rand(1, 4)

In [ ]:
MODEL_NAME = "pytorch"
LOCAL_MODEL_DIRECTORY = f"{LOCAL_MODEL_ARTIFACTS_DIR}/{MODEL_NAME}"
LOCAL_MODEL_VERSION_1 = f"{LOCAL_MODEL_DIRECTORY}/1"

%mkdir $LOCAL_MODEL_DIRECTORY
%mkdir $LOCAL_MODEL_VERSION_1

In [ ]:
LOCAL_MODEL_FILE = f"{LOCAL_MODEL_VERSION_1}/model.pt"

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(model, example)

# Save the TorchScript model
traced_script_module.save(LOCAL_MODEL_FILE)

#### 第六步。测试加载的TorchScript模型进行预测

In [ ]:
iris_classifier = torch.jit.load(LOCAL_MODEL_FILE)


def predict_class(instances):
    instances = torch.Tensor(instances)
    output = iris_classifier(instances)
    _, predicted = torch.max(output, 1)
    return predicted


predicted = predict_class(xtrain[0:5])
print([CLASS_VOCAB[class_index] for class_index in predicted])

准备 Triton 模型配置

[model 仓库](https://github.com/triton-inference-server/server/blob/main/docs/model_repository.md) 中的每个模型都必须包括一个[model 配置](https://github.com/triton-inference-server/server/blob/main/docs/model_configuration.md)，提供有关模型的必需和可选信息。Triton 后端是执行模型的实现，可以是深度学习框架的包装器。只要符合[后端 API](https://github.com/triton-inference-server/backend/blob/main/README.md#triton-backend-api)，后端还可以实现任何您想要的功能。Triton 使用这个 API 向后端发送执行请求，后端使用 API 与 Triton 通信。每个模型都必须与后端关联。模型的后端在模型配置中使用`backend`和`platform`设置来指定。

[要加载 PyTorch 模型](https://github.com/triton-inference-server/backend/blob/main/README.md#backends)，模型配置需要将`backend`和`platform`设置为`pytorch`和`pytorch_libtorch`。PyTorch 后端也有[特殊约定](https://github.com/triton-inference-server/server/blob/main/docs/model_configuration.md)。

In [ ]:
%%writefile $LOCAL_MODEL_DIRECTORY/config.pbtxt
backend: "pytorch"
platform: "pytorch_libtorch"
max_batch_size: 8
input {
    name: "INPUT__0"
    data_type: TYPE_FP32
    dims: [ 4 ]
}
output {
    name: "OUTPUT__0"
    data_type: TYPE_FP32
    dims: [ 1 ]
}

## 将模型工件上传到云存储

在您部署模型进行服务之前，Vertex AI 需要访问 Cloud Storage 中的以下文件：

* `model.pt`（模型工件）
* `config.pbtxt`（模型配置）

运行以下命令来上传您的文件：

In [ ]:
!gsutil cp -r {LOCAL_MODEL_ARTIFACTS_DIR}/* {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/
!gsutil ls {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/

使用NVIDIA Triton推理服务器

使用NVIDIA Triton推理服务器初始化本地模型。在本教程中，您将使用[NVIDIA Triton推理服务器](https://developer.nvidia.com/nvidia-triton-inference-server)来提供PyTorch模型。图像托管在`nvcr.io`上，并且所有发布的标签都可以在[此链接](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/tritonserver/tags)中找到。

In [ ]:
TRITON_VERSION = "21.08"

通过自定义预测程序功能，Vertex AI SDK允许您加载现有的图像进行本地测试，然后轻松部署到Vertex AI预测中。

要在Vertex AI上使用Triton推理服务器，您需要设置所需的参数，如预测路由、健康路由、端口和参数。默认情况下，Triton监听端口 `8000` 上的HTTP请求。可以在这里找到Triton的快速入门：[链接](https://github.com/triton-inference-server/server/blob/main/docs/quickstart.md#run-triton)。

In [ ]:
from google.cloud.aiplatform.prediction import LocalModel

local_model = LocalModel(
    serving_container_image_uri=f"nvcr.io/nvidia/tritonserver:{TRITON_VERSION}-py3",
    serving_container_predict_route=f"/v2/models/{MODEL_NAME}/infer",
    serving_container_health_route=f"/v2/models/{MODEL_NAME}",
    serving_container_ports=[8000],
    serving_container_args=["tritonserver", "--model-repository=$(AIP_STORAGE_URI)"],
)

你可以查看图片中的服务容器规格。

In [ ]:
local_model.get_serving_container_spec()

### 保存测试实例

要了解如何在JSON中格式化输入实例，[阅读文档。](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models#request-body-details)

In [ ]:
INPUT_FILE = "instances.json"

In [ ]:
%%writefile $INPUT_FILE
{
    "id": "0",
    "inputs": [
        {
            "name": "INPUT__0",
            "shape": [2, 4],
            "datatype": "FP32",
            "data": [[6.7, 3.1, 4.7, 1.5], [4.6, 3.1, 1.5, 0.2]]
        }
    ]
}

### 本地运行和测试容器

自定义预测例程提供两种方法来下载模型工件，以便在本地测试图像。
1. 本地路径。
2. GCS路径。

要使用本地路径，您的“Predictor”需要支持加载模型的两种方式，以便在本地进行测试并部署到Vertex AI预测服务。SDK提供了一个名为`download_model_artifacts`的方法在[prediction_utils.py](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/utils/prediction_utils.py)中支持这两种方式，您可以在`Predictor`中的`load`函数中调用该方法。

如果要使用GCS路径在本地测试图像，您需要按照说明设置凭据。

设置凭证

设置凭证仅在本地使用GCS路径运行自定义服务容器时才需要。设置凭证是执行“预测者”中的“加载”函数所必需的，该函数会从Google Cloud Storage下载模型工件。

要访问项目中的Google Cloud Storage，您需要通过以下方式之一设置凭证：

1.用户账号

2. 服务账号

您可以在[这里](https://cloud.google.com/docs/authentication#principals)了解更多关于以上每种方式的信息。

选项1. 使用Google用户凭据

首先授权Google auth库使用Google用户凭据访问Cloud平台。这一步涉及一个交互式提示，需要用户输入。如果您正在使用非交互式shell，您应该打开一个终端来运行这个命令。请参阅[这里](https://jupyterlab.readthedocs.io/en/stable/user/terminal.html)了解在Vertex Workbench笔记本环境中如何打开终端。

请注意，如果您在没有窗口管理器/浏览器的计算机上运行下面的命令，它会提示您运行命令`gcloud auth application-default login --remote-bootstrap="..."`。只有可以启动网络浏览器的计算机，例如笔记本电脑或工作站，才被允许运行这个命令。在使用浏览器完成验证步骤之后，您需要将验证代码复制粘贴回原始终端以继续登录流程。如果您在支持网络浏览器的计算机上运行这个命令，就不需要使用单独的计算机。

这个命令会将凭据文件打印为`Credentials saved to file: [CREDENTIALS_FILE]`。

In [ ]:
!gcloud auth application-default login

指定凭证文件，该文件是一个json文件，作为路径。

In [ ]:
CREDENTIALS_FILE = "[CREDENTIALS_FILE]"  # @param {type:"string"}

接下来，授权gcloud使用Google用户凭据访问Cloud平台。用户凭证需要在项目中具有`setIamPolicy`权限。有关更多详细信息，请参阅[此处](https://cloud.google.com/resource-manager/docs/access-control-proj)。

与之前的步骤类似，这也需要用户输入。之前步骤的说明同样适用于这里。

In [ ]:
!gcloud auth login

授予用户帐户角色。使用您在gcloud auth步骤中使用的电子邮件地址，例如`myemail@gmail.com`。由于您将使用此用户帐户从Google Cloud Storage下载模型工件，因此您授予用户帐户**Storage Object Viewer**角色。有关更多详细信息，请参阅[Cloud Storage的IAM角色](https://cloud.google.com/storage/docs/access-control/iam-roles)。

In [ ]:
USER_ACCOUNT = "[USER_ACCOUNT]"  # @param {type:"string"}

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=user:$USER_ACCOUNT --role=roles/storage.objectViewer

使用项目 ID 初始化 Vertex AI SDK，这是使用用户凭据所必需的。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

选项2. 使用Google服务账号凭据

如果您已经完成了上面的选项1，请跳过这一步。如果IAM API尚未启用，请首先启用它。

In [ ]:
!gcloud services enable iam.googleapis.com

服务账号

在本地运行容器时，您可以使用服务帐号来下载模型构件。如果您不想使用项目的计算引擎服务帐号，请将`SERVICE_ACCOUNT`设置为另一个服务帐号ID。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

创建服务账号。

In [ ]:
!gcloud iam service-accounts create $SERVICE_ACCOUNT

授权gcloud使用Google用户凭据访问Cloud Platform。用户凭据需要在项目中具有`setIamPolicy`权限。更多详情请参见[此链接](https://cloud.google.com/resource-manager/docs/access-control-proj)。

这一步涉及互动提示，需要用户输入。如果您正在使用非互动shell，您应该打开终端来运行此命令。请参见[此链接](https://jupyterlab.readthedocs.io/en/stable/user/terminal.html)了解如何在Vertex Workbench笔记本环境中打开终端。

请注意，如果您在没有窗口管理器/浏览器的机器上运行下面的命令，它将提示您运行命令`gcloud auth application-default login --remote-bootstrap="..."`。只有可以启动Web浏览器的机器，例如笔记本电脑或工作站，才允许运行此命令。完成使用浏览器的身份验证步骤后，您需要将验证代码复制并粘贴回原始终端以继续登录流程。如果您在有Web浏览器支持的机器上运行此命令，则无需使用单独的机器。

In [ ]:
!gcloud auth login

将角色授予服务帐户。由于您将使用此服务帐户从Google云存储中下载模型文件，因此您应授予服务帐户**Storage Object Viewer**角色。有关更多详情，请参阅[Cloud Storage的IAM角色](https://cloud.google.com/storage/docs/access-control/iam-roles)。

In [ ]:
!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:{SERVICE_ACCOUNT}@{PROJECT_ID}.iam.gserviceaccount.com \
    --role=roles/storage.objectViewer

指定服务账号密钥文件名。

In [ ]:
CREDENTIALS_FILE = "./credentials.json"

生成服务账号密钥文件。在本地运行自定义服务容器时将使用此文件。

In [ ]:
!gcloud iam service-accounts keys create $CREDENTIALS_FILE \
    --iam-account="{SERVICE_ACCOUNT}@{PROJECT_ID}.iam.gserviceaccount.com"

#### 在本地运行并发送请求到容器

通过自定义预测例程功能，可以很容易地在本地测试容器。

在这个例子中，容器执行一个预测请求和一个健康检查。

注意：第一次运行以下命令会有点慢，因为您需要从`nvcr.io`拉取图片。

选项1. 通过本地路径传递

该选项要求`Predictor`中的`load`函数支持从GCS路径和本地路径加载模型文件。

In [ ]:
with local_model.deploy_to_local_endpoint(
    artifact_uri=f"{LOCAL_MODEL_ARTIFACTS_DIR}",
) as local_endpoint:
    predict_response = local_endpoint.predict(
        request_file=INPUT_FILE,
        headers={"Content-Type": "application/json"},
    )

    health_check_response = local_endpoint.run_health_check()

打印出预测的响应以及其内容。

In [ ]:
print(predict_response, predict_response.content)

将健康检查的响应和内容打印出来。

In [ ]:
print(health_check_response, health_check_response.content)

也打印出所有容器日志。您将看到容器启动、处理请求和容器关闭的日志。

In [ ]:
local_endpoint.print_container_logs(show_all=True)

选项2. 传递GCS路径

如果你想传递GCS路径，你需要在之前的步骤中设置好凭证，并且在运行容器时传递路径给凭证。服务账户应该拥有**存储对象查看者**权限。

In [ ]:
with local_model.deploy_to_local_endpoint(
    artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
    credential_path=CREDENTIALS_FILE,
) as local_endpoint:
    predict_response = local_endpoint.predict(
        request_file=INPUT_FILE,
        headers={"Content-Type": "application/json"},
    )

    health_check_response = local_endpoint.run_health_check()

打印出预测的响应及其内容。

In [ ]:
print(predict_response, predict_response.content)

打印出健康检查的响应和内容。

In [ ]:
print(health_check_response, health_check_response.content)

也打印出所有容器日志。您将看到容器启动、请求处理和容器关闭的日志。

In [ ]:
local_endpoint.print_container_logs(show_all=True)

### 将图像URI更改为Artifact Registry

由于Vertex AI Prediction当前不支持`nvcr.io`存储库，因此您需要将图像复制到Artifact Registry。您可以使用复制的图像轻松启动另一个本地模型实例。

In [ ]:
local_model_ar = local_model.copy_image(
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}"
)

打印出新的Local 模型实例的服务容器规范。图像的 URI 应更改为 Artifact Registry。

In [ ]:
local_model_ar.get_serving_container_spec()

将容器推送到工件注册表

配置Docker以访问工件注册表。然后将您的容器镜像推送到您的工件注册表存储库。

打印出项目中所有已启用的服务。

In [ ]:
!gcloud services list

如果您的项目中未启用`artifactregistry.googleapis.com`，请在继续之前启用该API。

In [ ]:
!gcloud services enable artifactregistry.googleapis.com

In [ ]:
!gcloud artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$REGION

In [ ]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

使用SDK推送图片。

In [ ]:
local_model_ar.push_image()

部署到Vertex AI

### 上传NVIDIA Triton推理服务器模型

In [ ]:
from google.cloud import aiplatform

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

使用LocalModel实例来上传模型。它会自动为您填充容器规范。

In [ ]:
model = aiplatform.Model.upload(
    local_model=local_model_ar,
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
)

### 在 Vertex AI 上部署模型
完成此步骤后，模型已部署并准备好进行在线预测。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

发送预测

需要使用[原始预测](https://cloud.google.com/sdk/gcloud/reference/ai/endpoints/raw-predict)来发送任意有效负载。

使用REST

In [ ]:
ENDPOINT_ID = endpoint.name

In [ ]:
! curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
--data-binary @$INPUT_FILE \
https://{REGION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}:rawPredict

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
# Undeploy model and delete endpoint
endpoint.delete(force=True)

# Delete the model resource
model.delete()

# Delete the container image from Artifact Registry
!gcloud artifacts docker images delete \
    --quiet \
    --delete-tags \
    {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}

delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI